<a target="_blank" href="https://colab.research.google.com/github/thierrymoudiki/sktime/blob/nnetsauce-mts/examples/nnetsauce/2024-11-13-nnetsauce-MTS-example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
!pip uninstall sktime -y

In [ ]:
!pip install git+https://github.com/thierrymoudiki/sktime.git --upgrade --no-cache-dir

## Example 1

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import linear_model
from statsmodels.tsa.base.datetools import dates_from_str

from sktime.forecasting.nnetsaucemts import NnetsauceMTS

In [ ]:
# some example data
mdata = sm.datasets.macrodata.load_pandas().data
# prepare the dates index
dates = mdata[["year", "quarter"]].astype(int).astype(str)
quarterly = dates["year"] + "Q" + dates["quarter"]
quarterly = dates_from_str(quarterly)
mdata = mdata[["realgovt", "tbilrate", "cpi"]]
mdata.index = pd.DatetimeIndex(quarterly)
data = np.log(mdata).diff().dropna()

n = data.shape[0]
max_idx_train = np.floor(n * 0.9)
training_index = np.arange(0, max_idx_train)
testing_index = np.arange(max_idx_train, n)
df_train = data.iloc[training_index, :]
print(df_train.tail())
df_test = data.iloc[testing_index, :]

In [ ]:
# Adjust Ridge
regr = linear_model.Ridge()
obj_MTS = NnetsauceMTS(
    regr, lags=25, a=0.1, n_hidden_features=5, verbose=1, type_pi="gaussian"
)
display(obj_MTS.get_params())

In [ ]:
obj_MTS.fit(df_train)

In [ ]:
res = obj_MTS.predict(fh=[i for i in range(1, 10)])
print(res)

In [ ]:
res = obj_MTS.predict_quantiles(fh=[i for i in range(1, 10)], alpha=0.05)
print(res)

In [ ]:
obj_MTS.fitter.plot(series="realgovt")

In [ ]:
obj_MTS.fitter.plot(series="cpi")

## Example 2

In [ ]:
!pip install git+https://github.com/Techtonique/mlsauce.git --upgrade --no-cache-dir

In [ ]:
url = "https://raw.githubusercontent.com/Techtonique/"
url += "datasets/main/time_series/multivariate/"
url += "ice_cream_vs_heater.csv"
df_temp = pd.read_csv(url)
df_temp.index = pd.DatetimeIndex(df_temp.date)  # must have# first other difference
df_icecream = df_temp.drop(columns=["date"]).diff().dropna()

In [ ]:
import mlsauce as ms

regr = ms.RidgeRegressor(reg_lambda=100)
obj_MTS = NnetsauceMTS(
    regr,
    lags=25,
    n_hidden_features=3,
    verbose=1,
    type_pi="scp2-kde",
    replications=250,
    kernel="tophat",
)
display(obj_MTS.get_params())
obj_MTS.fit(df_icecream)
res = obj_MTS.predict(fh=[i for i in range(1, 25)])
display(res)
res = obj_MTS.predict_quantiles(fh=[i for i in range(1, 25)], alpha=0.10)
display(res)

In [ ]:
obj_MTS.fitter.plot(series="heater", type_plot="spaghetti")

In [ ]:
obj_MTS.fitter.plot(series="icecream", type_plot="spaghetti")